In [0]:
from datetime import datetime
from pyspark.sql import Row
import os

def log_event_file(
    pipeline_name,
    notebook_name,
    layer_name,
    status,
    message="",
    record_count=0,
    event_time=None
):

    # ---------- Set event time ----------
    if event_time is None:
        event_time = datetime.now()

    timestamp = event_time.strftime("%Y%m%d_%H%M%S")

    # ---------- Local log path (DBFS local storage) ----------
    # This stores logs in Databricks File System (acts like local system)
    local_log_dir = "/Volumes/workspace/default/patientinsurance_logfiles/"

    os.makedirs(local_log_dir, exist_ok=True)

    file_name = f"{pipeline_name}_{layer_name}_{timestamp}.log"
    full_local_path = f"{local_log_dir}{file_name}"

    # ---------- Log content ----------
    log_text = f"""
Event Time   : {event_time}
Pipeline     : {pipeline_name}
Notebook     : {notebook_name}
Layer        : {layer_name}
Status       : {status}
Record Count : {record_count}
Message      : {message}
"""

    # ---------- Write .log file locally ----------
    with open(full_local_path, "w") as f:
        f.write(log_text)

    # ---------- Write audit record to Delta table ----------
    audit_row = Row(
        event_time=event_time,
        pipeline_name=pipeline_name,
        notebook_name=notebook_name,
        layer_name=layer_name,
        status=status,
        record_count=record_count,
        message=message,
        log_file_path=full_local_path
    )

    spark.createDataFrame([audit_row]) \
        .write.mode("append") \
        .saveAsTable("audit_logs")

    print(f"Log created → {full_local_path}")


In [0]:
# %sql
# drop table audit_logs

In [0]:
# %sql
# select * from audit_logs order by event_time